In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Input, concatenate
from tensorflow.keras.models import Model

# Load the data
df = pd.read_csv('train_kaggle.csv')

# Extract the relevant columns
cols = ['Store_id', 'SKU_id', 'Date', 'Promo', 'Regular_Price', 'Promo_Price', 'Demand']
df = df[cols]

# Convert the 'Date' column to a datetime object
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)

# Create a pivot table with 'Store_id' and 'SKU_id' as the indices and 'Date' as the columns
# The values are the 'Demand' for each (store, SKU, date) combination
pivot_df = pd.pivot_table(df, values='Demand', index=['Store_id', 'SKU_id'], columns='Date', fill_value=0)

# Reshape the data into a 3-dimensional array
# The first dimension represents the samples (store-SKU pairs), the second dimension represents the time steps (days), and
# the third dimension represents the features (store ID, SKU ID, promo indicator, regular price, promo price)
X = pivot_df.reset_index().drop(['Store_id', 'SKU_id'], axis=1).values.reshape((-1, pivot_df.shape[1], 5))
y = pivot_df.values.reshape((-1, pivot_df.shape[1], 1))

# Split the data into training and validation sets
train_size = int(X.shape[0] * 0.8)
X_train, X_val = X[:train_size], X[train_size:]
y_train, y_val = y[:train_size], y[train_size:]

# Define the model architecture
inputs = Input(shape=(X.shape[1], X.shape[2]))
conv1 = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same')(inputs)
pool1 = MaxPooling1D(pool_size=2)(conv1)
conv2 = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same')(pool1)
pool2 = MaxPooling1D(pool_size=2)(conv2)
flatten = Flatten()(pool2)
dense1 = Dense(64, activation='relu')(flatten)
dense2 = Dense(y.shape[1])(dense1)
model = Model(inputs=inputs, outputs=dense2)
model.compile(optimizer='adam', loss='mse')

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_val, y_val))

# Use the model to make predictions
y_pred = model.predict(X_val)

# Compute the mean absolute error (MAE)
mae = np.mean(np.abs(y_val - y_pred))
print(f'MAE: {mae}')



In [2]:
!pip install tensorflow

     |████████████████████████████████| 244.3 MB 2.6 MB/s eta 0:00:01     |███████████████████████▏        | 176.6 MB 3.1 MB/s eta 0:00:23     |████████████████████████▋       | 187.6 MB 1.6 MB/s eta 0:00:36     |██████████████████████████▊     | 204.2 MB 2.0 MB/s eta 0:00:21
     |████████████████████████████████| 126 kB 4.6 MB/s eta 0:00:01
     |████████████████████████████████| 439 kB 4.1 MB/s eta 0:00:01
     |████████████████████████████████| 980 kB 5.4 MB/s eta 0:00:01
     |████████████████████████████████| 6.0 MB 3.5 MB/s eta 0:00:01
     |████████████████████████████████| 1.6 MB 21.7 MB/s eta 0:00:01
     |████████████████████████████████| 3.2 MB 4.6 MB/s eta 0:00:01
     |████████████████████████████████| 57 kB 4.8 MB/s eta 0:00:011
     |████████████████████████████████| 8.2 MB 1.9 MB/s eta 0:00:01
     |████████████████████████████████| 1.7 MB 2.0 MB/s eta 0:00:01
     |████████████████████████████████| 65 kB 2.4 MB/s eta 0:00:01
     |████████████████████████████████| 25.